# ELAIS-N1 Luminosity Function

Use the depth maps to get a histogram of areas with a given depth.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time
import glob

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table, join
from astropy.cosmology import FlatLambdaCDM



import numpy as np
from pymoc import MOC
import healpy as hp
#import pandas as pd #Astropy has group_by function so apandas isn't required.
import seaborn as sns

import warnings
#We ignore warnings - this is a little dangerous but a huge number of warnings are generated by empty cells later
warnings.filterwarnings('ignore')

from herschelhelp_internal.utils import inMoc, coords_to_hpidx, flux_to_mag, mag_to_flux
from herschelhelp_internal.masterlist import find_last_ml_suffix, nb_ccplots

from astropy.io.votable import parse_single_table

from pcigale.sed import SED
from pcigale.sed_modules import get_module

from lf.lf import volume, plot_LF

In [ ]:
os.environ['GAMA_DATA'] = 'We are not using GAMA data'
#from luminosity_function.gal_sample import CosmoLookup

In [ ]:
FIELD = 'ELAIS-N1'
FILTERS_DIR = "/opt/herschelhelp_python/database_builder/filters/"
DMU_DIR = '/mnt/hedam/dmu_products/'
#DMU_DIR = '/Users/rs548/GitHub/dmu_products/'

In [ ]:
depths = Table.read("{}dmu1/dmu1_ml_ELAIS-N1/data/depths_elais-n1_20180216.fits".format(DMU_DIR))
final_cat = Table.read("{}dmu32/dmu32_ELAIS-N1/data/ELAIS-N1_20171016.fits".format(DMU_DIR))

## I - Histogram of areas


In [ ]:
depths = depths["hp_idx_O_13", 
                "hp_idx_O_10", 
                "ferr_ap_irac_i1_mean", 
                "f_ap_irac_i1_p90", 
                "ferr_irac_i1_mean", 
                "f_irac_i1_p90"]

In [ ]:
depth_hist_plot = sns.distplot(depths["ferr_ap_irac_i1_mean"][~np.isnan(depths["ferr_ap_irac_i1_mean"])])
depth_hist_plot.set_xlim(0,5.)

In [ ]:
bins = np.linspace(0.,2.,1000)
depth_histogram = np.histogram(depths["ferr_ap_irac_i1_mean"][~np.isnan(depths["ferr_ap_irac_i1_mean"])], bins)

In [ ]:
np.max(depths["ferr_ap_irac_i1_mean"][~np.isnan(depths["ferr_ap_irac_i1_mean"])])

In [ ]:
depth_histogram[1][:-1]

In [ ]:
ax = sns.regplot(depth_histogram[1][:-1], depth_histogram[0])
ax.set(xlabel='ferr_ap_irac_i1_mean (uJ)', ylabel='N')

In [ ]:
Vmax = Table.read(DMU_DIR + 'dmu28/dmu28_ELAIS-N1/data/zphot/HELP_final_results.fits')#['id']
Vmax['id'].name = 'help_id'
Vmax = Vmax['help_id','UVoptIR_bayes.dust.luminosity', 'UVoptIR_bayes.dust.luminosity_err']

In [ ]:
Vmax[:10].show_in_notebook()

## II. First calaculate the irac_i1 flux as a function of redshift for each object

In [ ]:
#linearly spaced z - should this be logspace?
redshifts = np.linspace(0, 4, 100)
Vmax.add_column(Column(data=np.full((len(Vmax), len(redshifts)), 
                                    np.full(len(redshifts), np.nan)
                                   ) , 
                       name='f_z_relation'
                      )
               )

In [ ]:
Vmax[:10].show_in_notebook()

In [ ]:
redshifts = np.linspace(0, 4, 100)
n_absent = 0
n_processed = 0

for gal in Vmax['help_id']:

    try:
        orig_spec = Table.read("{}{}{}_best_model.fits".format(DMU_DIR, 
                                              'dmu28/dmu28_ELAIS-N1/data/zphot/best_model_fits/',
                                              gal
                                             ))
        
    except FileNotFoundError:
        n_absent += 1
        # print('fail')
        continue
        
    #print('{} no fail'.format(gal))
    s = SED()
    # This is wrong because the best SED we get from CIGALE is redshifted (written by Yannick)
    s.add_contribution("HELP_SED", orig_spec['wavelength'], orig_spec['L_lambda_total'])
    
    fluxes = []
    for r in redshifts:
        sed = s.copy()
        mod = get_module("redshifting", redshift=r)
        mod.process(sed)
        fluxes.append(sed.compute_fnu('IRAC1'))
        
    Vmax['f_z_relation'][Vmax['help_id'] == gal] = fluxes
        
    #print("{}:{}".format(gal,fluxes[0]))
    n_processed +=1

In [ ]:
print('{} processed and {} missing'.format(n_processed, n_absent))

## III. Then calculate the zmax for each depth bin for each object

Given the array of fluxes as a function of redhsift we interpolate at the depths based on taking the redshift that the object would have a flux equal to 5$\sigma$ for that mean error bin

In [ ]:
#Example object left over as the last object from the previous loop
np.interp(5 * bins *1.e-3, np.flip(fluxes,0), np.flip(redshifts,0))

In [ ]:
Vmax.add_column(Column(data=np.full((len(Vmax), len(bins)), 
                                    np.full(len(bins), np.nan)
                                   ) , 
                       name='zmax_histogram'
                      )
               )

In [ ]:
for gal in Vmax['help_id']:
    fluxes = np.array(Vmax[Vmax['help_id']==gal]['f_z_relation'])[0]
    z_max_f_relation = np.interp(5 * bins *1.e-3, np.flip(fluxes,0), np.flip(redshifts,0))
    Vmax['zmax_histogram'][Vmax['help_id'] == gal] = z_max_f_relation

Lets check the test object above is correct

In [ ]:
np.array(Vmax[Vmax['help_id']=='HELP_J155700.909+550039.328']['zmax_histogram'])[0]

## IV. Finally calculate the Vmax on EN1 for each object
For every object calculate the Vmax for each depth cell and multiply by the number of cells at that depth

In [ ]:
Vmax.add_column(Column(data=np.full((len(Vmax), len(bins)), 
                                    np.full(len(bins), np.nan)
                                   ) , 
                       name='vmax_histogram'
                      )
               )

In [ ]:
cosmo = FlatLambdaCDM(H0=100. , Om0 = (1-0.7))

#TODO: Make a lookup table for speed

In [ ]:
for gal in Vmax['help_id']:
    
    z_max_f_relation = np.array(Vmax[Vmax['help_id']==gal]['zmax_histogram'])[0]
    v_max_f_relation = cosmo.comoving_volume(z_max_f_relation)
    Vmax['vmax_histogram'][Vmax['help_id'] == gal] = v_max_f_relation.value
    
    

In [ ]:
Vmax.write("data/vmax_ELAIS-N1.fits", overwrite=True)

In [ ]:
#def gal_sample(z0,z1,Ldust,Vmax):
    